In [1]:
from reweighing import weighting_educ, weighting_maritalstatus

In [2]:
# applying weights
import csv
gender = []
educ = []
status = []
with open('adult.csv', newline='') as data:
    spamreader = csv.reader(data, delimiter=',')
    for row in spamreader:
        #[1:-1] because the words look like this: "'word'"
        educ.append(row[3])
        gender.append(row[9])
        status.append(row[5])
column_educ = weighting_educ(gender, educ)
column_marital = weighting_maritalstatus(gender, status)
print(column_educ, column_marital)

[1.332164446096795, 1.332164446096795, 0.8666866719740454, 0.47620514054249713, 0.8666866719740454, 1.332164446096795, 0.22697227205771683, 3.686201962181681, 1.332164446096795, 0.5160855483179206, 3.686201962181681, 0.5530070432417307, 2.475644100267446, 1.2312182755310215, 1.1635278373093116, 0.5530070432417307, 0.22697227205771683, 1.2312182755310215, 0.9997712313482513, 0.9703286623810337, 0.9703286623810337, 0.47620514054249713, 1.332164446096795, 0.5160855483179206, 0.6694390322716864, 0.9997712313482513, 0.22697227205771683, 0.22697227205771683, 0.22697227205771683, 0.22697227205771683, 0.5160855483179206, 0.47620514054249713, 0.5530070432417307, 0.9703286623810337, 0.6694390322716864, 3.686201962181681, 0.6694390322716864, 2.477595650675153, 3.686201962181681, 3.684772195023363, 0.9703286623810337, 3.684772195023363, 0.8666866719740454, 3.684772195023363, 0.6694390322716864, 2.475644100267446, 0.6694390322716864, 0.6694390322716864, 0.8666866719740454, 0.5160855483179206, 0.553